In [95]:
import os
import pandas

class riyun:
    def __init__(self,GZ):
        self.GZ=GZ    
        self.wx_tg={'甲':'木','乙':'木','丙':'火','丁':'火','戊':'土','己':'土','庚':'金','辛':'金','壬':'水','癸':'水'}
        self.wx_dz={'子':'水','丑':'土','寅':'木','卯':'木','辰':'土','巳':'火','午':'火','未':'土','申':'金','酉':'金','戌':'土','亥':'水'}
        self.wx={'甲':'阳木','乙':'阴木','丙':'阳火','丁':'阴火','戊':'阳土', \
                 '己':'阴土','庚':'阳金','辛':'阴金','壬':'阳水','癸':'阴水', \
                 '子':'阳水','丑':'阴土','寅':'阳木','卯':'阴木', \
                 '辰':'阳土','巳':'阳火','午':'阴火','未':'阴土', \
                 '申':'阳金','酉':'阴金','戌':'阳土','亥':'阴水'}   
        self.wxSheng=[['木','火'],['火','土'],['土','金'],['金','水'],['水','木']]
        self.wxKe=[['木','土'],['火','金'],['土','水'],['金','木'],['水','火']]
            
    #天干冲
    def tg_chong(self):
        h1=['甲','庚']
        h2=['乙','辛']
        h3=['丙','壬']
        h4=['丁','癸']
        H=[h1,h2,h3,h4]
        for h in H:
            if len(list(set(h).intersection(set(self.GZ))))==2:
                print(h[0],h[1],'冲')
                
    #天干克
    def tg_ke(self):
        #天干相克： 庚克甲，辛克乙，壬克丙，癸克丁。
        h1=['甲','庚']
        h2=['乙','辛']
        h3=['丙','壬']
        h4=['丁','癸']
        H=[h1,h2,h3,h4]
        for h in H:
            if len(list(set(h).intersection(set(self.GZ))))==2:
                print(h[1],'克',h[0])
                
    # 天干合
    def tg_he(self):
        h1=['甲','己']
        h2=['乙','庚']
        h3=['丙','辛']
        h4=['丁','壬']
        h5=['戊','癸']
        
        H=[h1,h2,h3,h4,h5]
        for h in H:
            if len(list(set(h).intersection(set(self.GZ))))==2:
                print(h[0],h[1],'合')
                
    #地支冲
    def dz_chong(self):
        #地支六冲： 子午相冲，卯酉相冲，寅申相冲， 已亥相冲，辰戌相冲，丑未相冲
        h1=['子','午']
        h2=['卯','酉']
        h3=['寅','申']
        h4=['巳','亥']
        h5=['辰','戌']
        h6=['丑','未']
        H=[h1,h2,h3,h4,h5,h6]
        for h in H:
            if len(list(set(h).intersection(set(self.GZ))))==2:
                print(h[0],h[1],'冲')
                
    #地支害
    def dz_hai(self):
        #地支相害:未相害 丑午相害 寅巳相害 卯辰相害 申亥相害 戌酉相害
        h1=['子','未']
        h2=['丑','午']
        h3=['寅','巳']
        h4=['卯','辰']
        h5=['申','亥']
        h6=['戌','酉']
        H=[h1,h2,h3,h4,h5,h6]
        for h in H:
            if len(list(set(h).intersection(set(self.GZ))))==2:
                print(h[0],h[1],'相害')
                
    #地支刑
    def dz_xing(self):
        #地支相刑:寅巳申三刑 丑戌未三刑 子卯相刑 辰、酉、午、亥自刑
        h1=['寅','巳','申']
        h2=['丑','戌','未']
        h3=['子','卯']

        H=[h1,h2]
        for h in H:
            if len(list(set(h).intersection(set(self.GZ))))==3:
                print(h[0],h[1],h[2],'三刑')
                
        if len(list(set(h3).intersection(set(self.GZ))))==2:
            print(h3[0],h3[1],'相刑')   
                
        c=[]
        for h in self.GZ:
            if h=='辰':
                c.append(h)
        if len(c)>=2:
            print('辰辰自刑')
            
        c=[]
        for h in self.GZ:
            if h=='酉':
                c.append(h)
        if len(c)>=2:
            print('酉酉自刑')
            
        c=[]
        for h in self.GZ:
            if h=='午':
                c.append(h)
        if len(c)>=2:
            print('午午自刑')
            
        c=[]
        for h in self.GZ:
            if h=='亥':
                c.append(h)
        if len(c)>=2:
            print('亥亥自刑')    
    
    #地支三合
    def dz_3he(self):
        #地支三合局： 申子辰三合水局，亥卯未三合木局， 寅午戌三合火局，巳酉丑三合金局
        h1=['申','子','辰']
        h2=['亥','卯','未']
        h3=['寅','午','戌']
        h4=['巳','酉','丑']

        H=[h1,h2,h3,h4]
        for h in H:
            if len(list(set(h).intersection(set(self.GZ))))==3:
                print(h[0],h[1],h[2],'三合')
    
    #地支六合
    def dz_6he(self):
        #地支六合： 子丑合土，寅亥合木，戌卯合火，辰酉合金，巳申合水，午未合火
        h1=['子','丑']
        h2=['寅','亥']
        h3=['戌','卯']
        h4=['辰','酉']
        h5=['巳','申']
        h6=['午','未']
        
        H=[h1,h2,h3,h4,h5,h6]
        for h in H:
            if len(list(set(h).intersection(set(self.GZ))))==2:
                print(h[0],h[1],'六合')
                
    #地支三会    
    def dz_3hui(self):
        #地支三会局： 申酉戌三会金局. 寅卯辰三会木局. 亥子丑三会水局. 巳午未三会火局.
        h1=['申','酉','戌']
        h2=['寅','卯','辰']
        h3=['亥','子','丑']
        h4=['巳','午','未']

        H=[h1,h2,h3,h4]
        for h in H:
            if len(list(set(h).intersection(set(self.GZ))))==3:
                print(h[0],h[1],h[2],'三会')
                
    #十神
    def ten_shen(self):
        todayGZ=[self.GZ[0],self.GZ[1],self.GZ[2],self.GZ[3],self.GZ[4],self.GZ[5]]
        rz=self.GZ[6] #日主
        
            
        rel=[]
        for gz in todayGZ:
            temp=[self.wx[rz][1],self.wx[gz][1]]
            print(gz,self.wx[gz][1])
            
            if self.wx[gz][1]==self.wx[rz][1]:
                if self.wx[gz][0]==self.wx[rz][0]:
                    print(self.wx[gz],'同',self.wx[rz],'比肩')
                    rel.append('比肩')
                else:
                    print(self.wx[gz],'同',self.wx[rz],'劫财')
                    rel.append('劫财')
            else:       
                for h in self.wxSheng:
                    if len(list(set(temp).intersection(set(h))))==2:
                        if h[0]==self.wx[rz][1]: #我生
                            if self.wx[gz][0]==self.wx[rz][0]: #同性
                                print(self.wx[rz],'生',self.wx[gz][0]+h[1],'食神')
                                rel.append('食神')
                            else: #异性
                                print(self.wx[rz],'生',self.wx[gz][0]+h[1],'伤官')
                                rel.append('伤官')
                                
                        if h[1]==self.wx[rz][1]: #生我
                            if self.wx[gz][0]==self.wx[rz][0]: #同性
                                print(self.wx[gz][0]+h[0],'生',self.wx[rz],'偏印')
                                rel.append('偏印')
                            else: #异性
                                print(self.wx[gz][0]+h[0],'生',self.wx[rz],'正印')
                                rel.append('正印')
                            
                for k in self.wxKe:
                    if len(list(set(temp).intersection(set(k))))==2:
                        if k[0]==self.wx[rz][1]: #我克
                            if self.wx[gz][0]==self.wx[rz][0]: #同性
                                print(self.wx[rz],'克',self.wx[gz][0]+k[1],'偏财')
                                rel.append('偏财')
                            else: #异性
                                print(self.wx[rz],'克',self.wx[gz][0]+k[1],'正财')
                                rel.append('正财')
                                
                        if k[1]==self.wx[rz][1]: #克我
                            if self.wx[gz][0]==self.wx[rz][0]: #同性
                                print(self.wx[gz][0]+k[0],'克',self.wx[rz],'七杀')
                                rel.append('七杀')
                            else: #异性
                                print(self.wx[gz][0]+k[0],'克',self.wx[rz],'正官')
                                rel.append('正官')
                    
        print(rel)

        
        
if __name__=='__main__':
    k=['壬','癸','辛','子','丁','戌','乙']
    today=riyun(k)
    today.ten_shen()
    
    

壬 水
阳水 生 阴木 正印
癸 水
阴水 生 阴木 偏印
辛 金
阴金 克 阴木 七杀
子 水
阳水 生 阴木 正印
丁 火
阴木 生 阴火 食神
戌 土
阴木 克 阳土 正财
['正印', '偏印', '七杀', '正印', '食神', '正财']


In [34]:
import os
k=['甲','子','辰','未','戊','辰','庚']
h1=['甲','己']


d=list(set(k).intersection(set(h1)))


print(len(d))

if "甲" in k and '己' in k:
    print('甲己合')
    
if "甲" in k and "庚" in k:
    print('甲庚冲')
    
    
print(os.getcwd())
    

    

1
甲庚冲
/home/jack/files/py/mypy


天干： 甲、乙、丙、丁、戊、已、庚、辛、壬、癸 
地支： 子、丑、寅、卯、辰、巳、午、未、申、酉、戌、亥

地支三会局：
申酉戌三会金局. 
寅卯辰三会木局. 
亥子丑三会水局. 
巳午未三会火局.

天干相冲：
甲庚冲，乙辛冲，丙壬冲，丁癸冲


天干相克：
庚克甲，辛克乙，壬克丙，癸克丁。

天干合化
甲己合化土，乙庚合化金，丙辛合化水，丁壬合化木，戊癸合化火

地支六冲：
子午相冲，卯酉相冲，寅申相冲， 已亥相冲，辰戌相冲，丑未相冲

地支相害
子未相害
丑午相害
寅巳相害
卯辰相害
申亥相害
戌酉相害

地支相刑
寅巳申三刑
丑戌未三刑
子卯相刑
辰、酉、午、亥自刑


地支三合局：
申子辰三合水局，亥卯未三合木局， 寅午戌三合火局，巳酉丑三合金局

地支六合：
子丑合土，寅亥合木，戌卯合火，辰酉合金，巳申合水，午未合火
五行四方：东方属木，南方属火，西方属金，北方属水，中央属土。

几种常见关系：

食神（伤官）驾杀、食伤生财、枭印夺食、比劫生食伤、比劫克财、财破印、财生官、官生印、印生比劫、墓神的关系（辰戌丑未四个库，对于天干地支相对应五行出现时候的影响）

（这些关系在同柱之间、在地支之间、在天干之间，都需要考虑）